In [6]:
from scipy.io import loadmat
wine = loadmat('wine.mat')
data = wine['data']
label = wine['labels']
testdata = wine['testdata']
testlabel = wine['testlabels']

In [7]:
print(data[0])
print(label[0])

[ 1.         -0.47687637 -0.47325129 -0.46939025  1.32678097 -0.32362082
  0.81431303  1.08979962  0.75105482  0.69463614 -0.70381277 -0.58602832]
[6]


In [8]:
len(data)

3249

In [9]:
import numpy as np   ##科学计算库 
import pandas as pd
import matplotlib.pyplot as plt  ##绘图库
import statsmodels.api as sm   

In [10]:
regr = sm.OLS(label,data)
res = regr.fit() 

In [11]:
res.params

array([ 5.82610034,  0.0685723 , -0.23103551, -0.0273828 ,  0.17082491,
       -0.0353694 ,  0.10740169, -0.13434188, -0.12987808,  0.06963982,
        0.13476528,  0.32021342])

In [12]:
y = np.dot(testdata,res.params)

In [13]:
testlabel.shape

(3248, 1)

In [14]:
y.resize(y.shape[0],1)

In [15]:
square = (y-testlabel)*(y-testlabel)
square.shape

(3248, 1)

In [16]:
squared_loss = np.sum(square,axis = 0)

In [17]:
print(squared_loss[0])

1753.2175175369225


In [18]:
data.shape[1]

12

In [19]:
new_data = data[:,[0,1,2,3]]

In [20]:
new_data.shape

(3249, 4)

In [21]:
def ordinary_least_square(label,data):
    regr = sm.OLS(label,data) ##OLS
    res = regr.fit()   ##outcome
    return res.params


def squared_loss(testdata,testlabel,params):
    y = np.dot(testdata,params)  ##test_data_label
    y.resize(y.shape[0],1)
    square = (y-testlabel)*(y-testlabel)  
    squared_loss = np.sum(square)   ##squared_loss of testdata
    return squared_loss

In [22]:
import math
lmin = math.inf
name = []
min_params = []
for i in range(1,data.shape[1]):
    for j in range(i+1,data.shape[1]):
        for k in range(j+1,data.shape[1]):
            new_data = data[:,[0,i,j,k]]
            params = ordinary_least_square(label,new_data)
            squaredloss = squared_loss(new_data,label,params)
            #print(squaredloss)
            if lmin > squaredloss:
                lmin = min(lmin,squaredloss)
                name = [0,i,j,k]
                print(name)
                min_params = params
new_testdata = testdata[:,name]
test_squareloss = squared_loss(new_testdata,testlabel,min_params)

[0, 1, 2, 3]
[0, 1, 2, 4]
[0, 1, 2, 7]
[0, 1, 2, 8]
[0, 1, 2, 11]
[0, 2, 3, 11]
[0, 2, 4, 11]
[0, 2, 8, 11]
[0, 2, 10, 11]


In [23]:
print(name)
print(test_squareloss)

[0, 2, 10, 11]
1807.6601059716302


In [24]:
temp_data = data[:,[0,2,8,11]]
temp_test_data = testdata[:,[0,2,8,11]]
temp_params = ordinary_least_square(label,temp_data)
a = squared_loss(temp_test_data,testlabel,temp_params)
a

1795.4455371330366

In [25]:
cor = pearsonr(testdata[:,2],testdata[:,3])
print(cor)

(-0.3755630209692122, 2.5269603965354535e-109)


In [28]:
from scipy.stats import pearsonr

name_value = []
for i in range(1,len(name)):
    count = 1
    nv_list = []
    for j in range(1,testdata.shape[1]):
        cor = pearsonr(testdata[:,name[i]],testdata[:,j])
        if name[i] == j:
            continue
        elif count == 1:
            nv_list.append(j)
            nv_list.append(cor[0])
            count += 1
        elif count == 2:
            if abs(cor[0]) > abs(nv_list[1]):
                nv_list.insert(0,j)
                nv_list.insert(1,cor[0])
            else:
                nv_list.append(j)
                nv_list.append(cor[0])
            count += 1
        elif count > 2:
            if abs(cor[0]) > abs(nv_list[1]):
                nv_list.insert(0,j)
                nv_list.insert(1,cor[0])
                del nv_list[4:]
            elif abs(cor[0]) < abs(nv_list[1]) and abs(cor[0]) > abs(nv_list[3]):
                nv_list[2] = j
                nv_list[3] = cor[0]

    name_value.append(nv_list)
print(name_value)

[[7, -0.4220072736318842, 3, -0.3755630209692122], [5, 0.4071152150059981, 1, 0.29686322762159284], [8, -0.6795934959123543, 4, -0.34626243022889536]]


In [34]:
import numpy as np   ##科学计算库 
import pandas as pd
import matplotlib.pyplot as plt  ##绘图库
import statsmodels.api as sm   ##引入最小二乘法算法
from scipy.io import loadmat
import math
from scipy.stats import pearsonr





def ordinary_least_square(label,data):
    regr = sm.OLS(label,data) ##OLS
    res = regr.fit()   ##outcome
    return res.params


def squared_loss(testdata,testlabel,params):
    y = np.dot(testdata,params)  ##test_data_label
    y.resize(y.shape[0],1)
    square = (y-testlabel)*(y-testlabel)  
    squared_loss = np.sum(square,axis = 0)   ##squared_loss of testdata
#     print(squared_loss)
    n = testdata.shape[0]
    return squared_loss/n


def sparse_linear_predictor(data,label,testdata,testlabel):
    lmin = math.inf
    name = []
    min_params = []
    for i in range(1,data.shape[1]):
        for j in range(i+1,data.shape[1]):
            for k in range(j+1,data.shape[1]):
                new_data = data[:,[0,i,j,k]]
                params = ordinary_least_square(label,new_data)
                squaredloss = squared_loss(new_data,label,params)
#                 print(squaredloss)
                if lmin > squaredloss:
                    lmin = min(lmin,squaredloss)
                    name = [0,i,j,k]
#                     print(name)
                    min_params = params
    new_testdata = testdata[:,name]
    test_squareloss = squared_loss(new_testdata,testlabel,min_params)
    return name, min_params, test_squareloss


def pearson_correlation(testdata,name):
    name_value = []
    for i in range(1,len(name)):
        count = 1
        nv_list = []
        for j in range(1,testdata.shape[1]):
            cor = pearsonr(testdata[:,name[i]],testdata[:,j])
            if name[i] == j:
                continue
            elif count == 1:
                nv_list.append(j)
                nv_list.append(cor[0])
                count += 1
            elif count == 2:
                if abs(cor[0]) > abs(nv_list[1]):
                    nv_list.insert(0,j)
                    nv_list.insert(1,cor[0])
                else:
                    nv_list.append(j)
                    nv_list.append(cor[0])
                count += 1
            elif count > 2:
                if abs(cor[0]) > abs(nv_list[1]):
                    nv_list.insert(0,j)
                    nv_list.insert(1,cor[0])
                    del nv_list[4:]
                elif abs(cor[0]) < abs(nv_list[1]) and abs(cor[0]) > abs(nv_list[3]):
                    nv_list[2] = j
                    nv_list[3] = cor[0]

        name_value.append(nv_list)
    return name_value



if __name__ == '__main__':
    wine = loadmat('wine.mat')
    data = wine['data']
    label = wine['labels']
    testdata = wine['testdata']
    testlabel = wine['testlabels']

    params = ordinary_least_square(label,data)
    squareloss = squared_loss(testdata,testlabel,params)
    print("Test risks of the ordinary least squares estimator", squareloss)

    res = sparse_linear_predictor(data,label,testdata,testlabel)
    print("Test risks of the sparse linear predictor", res[2])
    print("Names_ID and value",res[0],res[1])

    name_value = pearson_correlation(testdata,res[0])
    print("nameID_value",name_value)

Test risks of the ordinary least squares estimator [0.53978372]
Test risks of the sparse linear predictor [0.5565456]
Names_ID and value [0, 2, 10, 11] [ 5.82610034 -0.24474264  0.10847122  0.37362602]
nameID_value [[7, -0.4220072736318842, 3, -0.3755630209692122], [5, 0.4071152150059981, 1, 0.29686322762159284], [8, -0.6795934959123543, 4, -0.34626243022889536]]


In [1]:
import numpy as np   ##科学计算库 
import pandas as pd
import matplotlib.pyplot as plt  ##绘图库
import statsmodels.api as sm   ##引入最小二乘法算法
from scipy.io import loadmat
import math
from scipy.stats import pearsonr





def ordinary_least_square(label,data):
    regr = sm.OLS(label,data) ##OLS
    res = regr.fit()   ##outcome
    return res.params


def squared_loss(testdata,testlabel,params):
    y = np.dot(testdata,params)  ##test_data_label
    y.resize(y.shape[0],1)
    square = (y-testlabel)*(y-testlabel)  
    squared_loss = np.sum(square,axis = 0)   ##squared_loss of testdata
#     print(squared_loss)
    n = testdata.shape[0]
    return squared_loss/n


def sparse_linear_predictor(data,label,testdata,testlabel):
    lmin = math.inf
    name = []
    min_params = []
    for i in range(1,data.shape[1]):
        for j in range(i+1,data.shape[1]):
            for k in range(j+1,data.shape[1]):
                new_data = data[:,[0,i,j,k]]
                params = ordinary_least_square(label,new_data)
                squaredloss = squared_loss(new_data,label,params)
#                 print(squaredloss)
                if lmin > squaredloss:
                    lmin = min(lmin,squaredloss)
                    name = [0,i,j,k]
#                     print(name)
                    min_params = params
    new_testdata = testdata[:,name]
    test_squareloss = squared_loss(new_testdata,testlabel,min_params)
    return name, min_params, test_squareloss


def pearson_correlation(testdata,name):
    name_value = []
    for i in range(1,len(name)):
        count = 1
        nv_list = []
        for j in range(1,testdata.shape[1]):
            cor = pearsonr(testdata[:,name[i]],testdata[:,j])
            if name[i] == j:
                continue
            elif count == 1:
                nv_list.append(j)
                nv_list.append(cor[0])
                count += 1
            elif count == 2:
                if abs(cor[0]) > abs(nv_list[1]):
                    nv_list.insert(0,j)
                    nv_list.insert(1,cor[0])
                else:
                    nv_list.append(j)
                    nv_list.append(cor[0])
                count += 1
            elif count > 2:
                if abs(cor[0]) > abs(nv_list[1]):
                    nv_list.insert(0,j)
                    nv_list.insert(1,cor[0])
                    del nv_list[4:]
                elif abs(cor[0]) < abs(nv_list[1]) and abs(cor[0]) > abs(nv_list[3]):
                    nv_list[2] = j
                    nv_list[3] = cor[0]

        name_value.append(nv_list)
    return name_value



if __name__ == '__main__':
    wine = loadmat('wine.mat')
    data = wine['data']
    label = wine['labels']
    testdata = wine['testdata']
    testlabel = wine['testlabels']

    params = ordinary_least_square(label,data)
    squareloss = squared_loss(testdata,testlabel,params)
    print("Test risks of the ordinary least squares estimator", squareloss)

    res = sparse_linear_predictor(data,label,testdata,testlabel)
    print("Test risks of the sparse linear predictor", res[2])
    print("Names_ID and value",res[0],res[1])

    name_value = pearson_correlation(testdata,res[0])
    print("nameID_value",name_value)

Test risks of the ordinary least squares estimator [0.53978372]
Test risks of the sparse linear predictor [0.5565456]
Names_ID and value [0, 2, 10, 11] [ 5.82610034 -0.24474264  0.10847122  0.37362602]
nameID_value [[7, -0.4220072736318842, 3, -0.3755630209692122], [5, 0.4071152150059981, 1, 0.29686322762159284], [8, -0.6795934959123543, 4, -0.34626243022889536]]


In [2]:
import numpy as np   ##科学计算库 
import pandas as pd
import matplotlib.pyplot as plt  ##绘图库
import statsmodels.api as sm   ##引入最小二乘法算法
from scipy.io import loadmat
import math
from scipy.stats import pearsonr





def ordinary_least_square(label,data):
    regr = sm.OLS(label,data) ##OLS
    res = regr.fit()   ##outcome
    return res.params


def squared_loss(testdata,testlabel,params):
    y = np.dot(testdata,params)  ##test_data_label
    y.resize(y.shape[0],1)
    square = (y-testlabel)*(y-testlabel)  
    squared_loss = np.sum(square,axis = 0)   ##squared_loss of testdata
#     print(squared_loss)
    n = testdata.shape[0]
    return squared_loss/n


def sparse_linear_predictor(data,label,testdata,testlabel):
    lmin = math.inf
    name = []
    min_params = []
    for i in range(1,data.shape[1]):
        for j in range(i+1,data.shape[1]):
            for k in range(j+1,data.shape[1]):
                new_data = data[:,[0,i,j,k]]
                params = ordinary_least_square(label,new_data)
                squaredloss = squared_loss(new_data,label,params)
#                 print(squaredloss)
                if lmin > squaredloss:
                    lmin = min(lmin,squaredloss)
                    name = [0,i,j,k]
#                     print(name)
                    min_params = params
    new_testdata = testdata[:,name]
    test_squareloss = squared_loss(new_testdata,testlabel,min_params)
    return name, min_params, test_squareloss


def pearson_correlation(testdata,name):
    name_value = []
    for i in range(1,len(name)):
        count = 1
        nv_list = []
        for j in range(1,testdata.shape[1]):
            cor = pearsonr(testdata[:,name[i]],testdata[:,j])
            if name[i] == j:
                continue
            elif count == 1:
                nv_list.append(j)
                nv_list.append(cor[0])
                count += 1
            elif count == 2:
                if abs(cor[0]) > abs(nv_list[1]):
                    nv_list.insert(0,j)
                    nv_list.insert(1,cor[0])
                else:
                    nv_list.append(j)
                    nv_list.append(cor[0])
                count += 1
            elif count > 2:
                if abs(cor[0]) > abs(nv_list[1]):
                    nv_list.insert(0,j)
                    nv_list.insert(1,cor[0])
                    del nv_list[4:]
                elif abs(cor[0]) < abs(nv_list[1]) and abs(cor[0]) > abs(nv_list[3]):
                    nv_list[2] = j
                    nv_list[3] = cor[0]

        name_value.append(nv_list)
    return name_value



if __name__ == '__main__':
    wine = loadmat('wine.mat')
    data = wine['data']
    label = wine['labels']
    testdata = wine['testdata']
    testlabel = wine['testlabels']

    params = ordinary_least_square(label,data)
    squareloss = squared_loss(testdata,testlabel,params)
    print("Test risks of the ordinary least squares estimator", squareloss)

    res = sparse_linear_predictor(data,label,testdata,testlabel)
    print("Test risks of the sparse linear predictor", res[2])
    print("Names_ID and value",res[0],res[1])

    name_value = pearson_correlation(testdata,res[0])
    print("nameID_value",name_value)

Test risks of the ordinary least squares estimator [0.53978372]
Test risks of the sparse linear predictor [0.5565456]
Names_ID and value [0, 2, 10, 11] [ 5.82610034 -0.24474264  0.10847122  0.37362602]
nameID_value [[7, -0.4220072736318842, 3, -0.3755630209692122], [5, 0.4071152150059981, 1, 0.29686322762159284], [8, -0.6795934959123543, 4, -0.34626243022889536]]
